In [2]:
from dataset import SyntheticSlateBanditDataset

In [3]:
# settings

random_state=12345
n_rounds=100
dim_context=10
n_actions=15
len_list=3

In [4]:
dataset = SyntheticSlateBanditDataset(
    n_unique_action=n_actions,
    len_list=len_list,
    dim_context=dim_context
)
bandit_feedback = dataset.obtain_batch_bandit_feedback(n_rounds=n_rounds)
print(bandit_feedback)

{'n_rounds': 100, 'n_unique_action': 15, 'context': array([[-2.04707659e-01,  4.78943338e-01, -5.19438715e-01,
        -5.55730304e-01,  1.96578057e+00,  1.39340583e+00,
         9.29078767e-02,  2.81746153e-01,  7.69022568e-01,
         1.24643474e+00],
       [ 1.00718936e+00, -1.29622111e+00,  2.74991633e-01,
         2.28912879e-01,  1.35291684e+00,  8.86429341e-01,
        -2.00163731e+00, -3.71842537e-01,  1.66902531e+00,
        -4.38569736e-01],
       [-5.39741446e-01,  4.76985010e-01,  3.24894392e+00,
        -1.02122752e+00, -5.77087303e-01,  1.24121276e-01,
         3.02613562e-01,  5.23772068e-01,  9.40277775e-04,
         1.34380979e+00],
       [-7.13543985e-01, -8.31153539e-01, -2.37023165e+00,
        -1.86076079e+00, -8.60757398e-01,  5.60145293e-01,
        -1.26593449e+00,  1.19827125e-01, -1.06351245e+00,
         3.32882716e-01],
       [-2.35941881e+00, -1.99542955e-01, -1.54199553e+00,
        -9.70735912e-01, -1.30703025e+00,  2.86349747e-01,
         3.7798411

In [5]:
# health　check
bandit_feedback["marginal_pi_b_at_position"][0].sum(1)

array([1., 1., 1.])

In [6]:
bandit_feedback["ranking_pi_b"].sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])